## Suppl. Create a table of z-scores


In [1]:
import glob 
import numpy as np
import pandas as pd
import nibabel as nib
from numpy import savetxt
from numpy import genfromtxt
from nilearn import plotting
import matplotlib.pyplot as plt
import nilearn.plotting as plotting
from nilearn.maskers import NiftiSpheresMasker

### Loop through HRF

In [55]:
# --- Init variables --- 

analysis_folder = '/project/4180000.19/multirat_stim/scratch/rabies_test/'
first_level_path = analysis_folder+'first_level/'

hrfs = ['Two_gammas', 'Peak_span', 'glover', 'spm', 'block']  
hrf_paths = {'Two_gammas': analysis_folder+'first_level/Two_gammas/mask_ROIs/',
            'Peak_span': analysis_folder+'first_level/Peak_span/mask_ROIs/',
            'glover' : analysis_folder+'first_level/glover/mask_ROIs/', 
            'spm' : analysis_folder+'first_level/spm/mask_ROIs/',
            'block' : analysis_folder+'first_level/block/mask_ROIs/'} 

### Group level zscores

In [61]:
import pandas as pd

hrfs = ['Peak_span']  

# Initialize lists to store the results
all_avg_zscores = []
all_std_zscores = []
zscores_df = []

for hrf in hrfs:
    print('Processing HRF:', hrf)
    hrf_path = hrf_paths[hrf]

    avg_zscores_list = []
    std_zscores_list = []

    for ds_num in range(1, 2):
        all_zscores = []
        ds = f'{ds_num:02d}'

        for subj_num in range(0, 10):
            try:
                zscores_csv = pd.read_csv(hrf_path + f'z_scores/zscores_sub-020{ds}0{subj_num}_ses-1.csv', header=None)
                all_zscores.append(zscores_csv)
            except FileNotFoundError:
                continue
        
        print('all_zscores', all_zscores)
        average_zscores = pd.concat(all_zscores).mean()
        std_zscores = pd.concat(all_zscores).std()
        #print(std_zscores)
        
        # Append the results to the respective lists
        avg_zscores_list.append(average_zscores.values[0])
        std_zscores_list.append(std_zscores.values[0])
        
    
    # Create DataFrames for average and std z-scores for this HRF
    df_avg = pd.DataFrame({'Average Z-Scores': avg_zscores_list}, index=[f'ds_{i:02d}' for i in range(1, 23)])
    df_std = pd.DataFrame({'Standard Deviation': std_zscores_list}, index=[f'ds_{i:02d}' for i in range(1, 23)])
    
    df_avg.to_csv(f'/project/4180000.19/multirat_stim/scratch/rabies_test/second_level/{hrf}/mean_zscores/mean_zscores_ds-{ds}.csv', header=False, index=True)
    df_std.to_csv(f'/project/4180000.19/multirat_stim/scratch/rabies_test/second_level/{hrf}/mean_zscores/std-deviation_zscores_ds-{ds}.csv', header=False, index=True)

    # Combine the DataFrames
    combined_df = pd.concat([df_avg, df_std], axis=1)
    
    # Append the combined DataFrame for this HRF to the list
    zscores_df.append(combined_df)

final_df = pd.concat(zscores_df, keys=hrfs, names=['HRF', 'Dataset'])
final_df.to_csv(f'/project/4180000.19/multirat_stim/scratch/plots/mean_zscores/HRFs_mean-std_zscores.csv')

print(final_df)

Processing HRF: Peak_span
all_zscores [          0
0  8.481281,           0
0  1.575252,           0
0  3.982688,           0
0  8.068061,           0
0  3.193054,           0
0  1.421595,           0
0  3.639332,          0
0  2.57244,          0
0  2.23472,          0
0 -0.12213]
                   Average Z-Scores  Standard Deviation
HRF       Dataset                                      
Peak_span ds_01            3.504629            2.781532
          ds_02            3.504629            2.781532
          ds_03            3.504629            2.781532
          ds_04            3.504629            2.781532
          ds_05            3.504629            2.781532
          ds_06            3.504629            2.781532
          ds_07            3.504629            2.781532
          ds_08            3.504629            2.781532
          ds_09            3.504629            2.781532
          ds_10            3.504629            2.781532
          ds_11            3.504629          

### Individual level zscores

In [60]:
zscores_df = pd.DataFrame()  # Initialize an empty DataFrame to store all z-scores

for hrf in hrfs:                                         # Loop through HRFs
    print('Processing HRF:', hrf)
    hrf_path = hrf_paths[hrf]
    hrf_zscores = pd.DataFrame()                         # Initialize a DataFrame for each HRF

    for ds_num in range(1, 23):                           # Loop through datasets
        all_zscores = []                                  # Initialize list to store z-scores for all subjects in the dataset
        ds = f'{ds_num:02d}'

        for subj_num in range(0, 10):                    # Loop through subjects within the dataset
            try:
                # Read the z-scores file for the current subject
                zscores_csv = pd.read_csv(hrf_path + f'z_scores/zscores_sub-020{ds}0{subj_num}_ses-1.csv', header=None)
                all_zscores.append(zscores_csv)
            except FileNotFoundError:
                continue

        if all_zscores:  # If there are z-scores to process
            combined_zscores = pd.concat(all_zscores, axis=1)  # Concatenate all z-scores for the dataset across subjects

            # Flatten and create a series with unique column names for each z-score in the dataset
            combined_zscores.columns = [f'{hrf}_ds-{ds}_subj-{subj_num}' for subj_num in range(combined_zscores.shape[1])]

            # Append combined z-scores to the hrf_zscores DataFrame for this HRF
            hrf_zscores = pd.concat([hrf_zscores, combined_zscores], axis=1)

    # Merge with the overall zscores_df on index to keep alignment of rows/columns
    zscores_df = pd.concat([zscores_df, hrf_zscores], axis=1)

# Save the resulting DataFrame to CSV
zscores_df.to_csv(f'/project/4180000.19/multirat_stim/scratch/plots/mean_zscores/HRFs_individual_zscores.csv')

# Display the final zscores_df
print(zscores_df)


Processing HRF: Two_gammas
Processing HRF: Peak_span
Processing HRF: glover
Processing HRF: spm
Processing HRF: block
   Two_gammas_ds-01_subj-0  Two_gammas_ds-01_subj-1  Two_gammas_ds-01_subj-2  \
0                 4.360135                 0.327204                 1.305259   

   Two_gammas_ds-01_subj-3  Two_gammas_ds-01_subj-4  Two_gammas_ds-01_subj-5  \
0                 3.384516                 2.228767                 0.390096   

   Two_gammas_ds-01_subj-6  Two_gammas_ds-01_subj-7  Two_gammas_ds-01_subj-8  \
0                 1.758436                 0.944008                 0.906288   

   Two_gammas_ds-01_subj-9  ...  block_ds-22_subj-0  block_ds-22_subj-1  \
0                -0.013368  ...           -4.923715           -1.752598   

   block_ds-22_subj-2  block_ds-22_subj-3  block_ds-22_subj-4  \
0           -0.634145            -4.66322            2.381456   

   block_ds-22_subj-5  block_ds-22_subj-6  block_ds-22_subj-7  \
0            -0.95078           -4.024012           